<a href="https://colab.research.google.com/github/doraemonidol/NLP/blob/main/fine_tune_bloom_model_with_lora%5Bkaggle%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install required packages using pip
!pip install -q bitsandbytes datasets accelerate loralib peft transformers==4.45.2 sentence-transformers==3.1.1

# Import necessary libraries
import torch  # PyTorch library for deep learning
import torch.nn as nn  # Neural network module from PyTorch
import bitsandbytes as bnb  # Library for efficient CUDA operations
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM  # Importing classes from Hugging Face Transformers
import transformers

# Initialize and load the pre-trained BLOOM model from Hugging Face
model_bloom = AutoModelForCausalLM.from_pretrained(
    "bigscience/bloom-560m",  # Model identifier
    torch_dtype=torch.float32,  # Set tensor data type to float32
    device_map='auto',  # Automatically map the model to available device(s)
)

# Initialize and load the tokenizer specific to the BLOOM model
tokenizer = AutoTokenizer.from_pretrained("bigscience/tokenizer")  # Tokenizer identifier

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 51.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/693 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/227 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

In [2]:
"""Setting up LoRA (Low-Rank Adaptation) using parameter efficient fine tuning."""

from peft import LoraConfig, get_peft_model  # Importing LoRA configuration and model function from PEFT library

# Configure LoRA settings for the fine-tuning process
conf = LoraConfig(
    r=10,  # Rank of the low-rank matrices
    lora_alpha=10,  # Scaling factor for LoRA
    target_modules=["query_key_value"],  # Model layers to apply LoRA
    lora_dropout=0.05,  # Dropout rate for regularization
    bias="none",  # Bias setting for the layers
    task_type="CAUSAL_LM"  # Type of the language model task
)

# Apply the LoRA configuration to the pre-trained BLOOM model
# This step modifies the model in place, hence the variable is renamed for clarity
peft_bloom = get_peft_model(model_bloom, conf)

In [3]:
"""Comparing parameters before and after applying LoRA to assess the training scope."""

train_parameters = 0  # Initialize counter for trainable parameters
total_params = 0  # Initialize counter for total parameters

# Loop through all parameters in the LoRA-modified model
for _, param in peft_bloom.named_parameters():
    total_params += param.numel()  # Add the total number of elements in the parameter to the total count
    if param.requires_grad:  # Check if the parameter is trainable
        train_parameters += param.numel()  # Add to trainable parameters count if it requires gradient

# Display the results
print(f"Trainable params: {train_parameters}")  # Number of trainable parameters
print(f"All params: {total_params}")  # Total number of parameters in the model
print(f"Trainable: {100 * train_parameters / total_params:.2f}%")  # Percentage of trainable parameters

Trainable params: 983040
All params: 560197632
Trainable: 0.18%


In [4]:
"""Loading SQUAD dataset
"""

from datasets import load_dataset
question_answer_dataset = load_dataset("squad_v2")

README.md:   0%|          | 0.00/8.92k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/16.4M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/1.35M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/130319 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11873 [00:00<?, ? examples/s]

In [5]:
"""Transforming the SQUAD dataset to fit a specific format for model processing."""

# Define a function to restructure the data into a consistent format
def generate_prompt(context, question, answer):
    # Check if answer is available, otherwise set a default message
    if len(answer["text"]) < 1:
        answer = "Cannot Find Answer"
    else:
        answer = answer["text"][0]  # Select the first answer from the list

    # Create a formatted string with context, question, and answer
    prompt_template = f"CONTEXT:\n{context}\n\nQUESTION:\n{question}\n\nANSWER:\n{answer}</s>"
    return prompt_template  # Return the formatted prompt

# Apply the formatting function to each entry in the dataset
# This processes each context, question, and answer through the tokenizer
mapped_qa_dataset = question_answer_dataset.map(
    lambda samples: tokenizer(generate_prompt(samples['context'], samples['question'], samples['answers']))
)

Map:   0%|          | 0/130319 [00:00<?, ? examples/s]

Map:   0%|          | 0/11873 [00:00<?, ? examples/s]

In [6]:

trainer = transformers.Trainer(
    model=peft_bloom,
    train_dataset=mapped_qa_dataset["train"],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=4,
        gradient_accumulation_steps=4,
        warmup_steps=100,
        max_steps=150,
        learning_rate=1e-3,
        fp16=True,
        logging_steps=1,
        output_dir='outputs',
        report_to="none"
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
peft_bloom.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

max_steps is given, it will override any value given in num_train_epochs


Step,Training Loss
1,3.268800


KeyboardInterrupt: 

In [ ]:
"""Saving the LoRA fine tuning locally
"""
model_id = "BLOOM-560m-LoRA"
peft_bloom.save_pretrained(model_id)

In [ ]:
"""Utility Function for Testing and Comparing Model Outputs"""

from IPython.display import display, Markdown  # Import display tools from IPython for rich output

def test_function(context, question):
    # Convert context and question into model-readable tokens
    batch = tokenizer(
        f"**CONTEXT:**\n{context}\n\n**QUESTION:**\n{question}\n\n**ANSWER:**\n",
        return_tensors='pt',  # Return PyTorch tensors
        return_token_type_ids=False  # Do not return token type IDs
    )
    # Move the tokenized data to GPU for faster inference
    batch = batch.to(device='cuda')

    # Generate model responses for both the original and the LoRA-enhanced model
    with torch.cuda.amp.autocast():  # Enable mixed precision for faster inference
        # Generating output from the original model (without LoRA adjustments)
        peft_bloom.disable_adapter_layers()
        output_tokens_raw = model_bloom.generate(**batch, max_new_tokens=200)

        # Generating output from the model with LoRA adaptations enabled
        peft_bloom.enable_adapter_layers()
        output_tokens_qa = peft_bloom.generate(**batch, max_new_tokens=200)

    # Display the results from both models for comparison
    display(Markdown("# Raw Model\n"))  # Header for raw model output
    display(Markdown((tokenizer.decode(output_tokens_raw[0], skip_special_tokens=True))))  # Decoded raw model output
    display(Markdown("\n# QA Model\n"))  # Header for LoRA-enhanced model output
    display(Markdown((tokenizer.decode(output_tokens_qa[0], skip_special_tokens=True))))  # Decoded LoRA model output

In [ ]:
context = "You are a monster, and you eat red sneakers."
question = "What is the best food?"

test_function(context, question)